In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('amazon_products.csv')
data.shape

(1426337, 11)

In [5]:
titles = data[['title']].dropna().drop_duplicates()
titles.shape
print(titles['title'][:10])

0    Sion Softside Expandable Roller Luggage, Black...
1    Luggage Sets Expandable PC+ABS Durable Suitcas...
2    Platinum Elite Softside Expandable Checked Lug...
3    Freeform Hardside Expandable with Double Spinn...
4    Winfield 2 Hardside Expandable Luggage with Sp...
5    Maxlite 5 Softside Expandable Luggage with 4 S...
6    Hard Shell Carry on Luggage Airline Approved, ...
7    Maxporter II 30" Hardside Spinner Trunk Luggag...
8    Omni 2 Hardside Expandable Luggage with Spinne...
9    Luggage Sets Expandable Lightweight Suitcases ...
Name: title, dtype: object


In [6]:
import re

# 1. Clean punctuation from each title
titles['clean_title'] = titles['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# 2. Split and flatten all words into one list
all_words = []
for title in titles['clean_title']:
    if pd.notna(title):  # make sure title is not NaN
        words = title.split()
        all_words.extend(words)

# 3. Get unique words
unique_words = list(set(all_words))

# 4. Sort them if you want
unique_words = sorted(unique_words)

print(len(unique_words))

830546


| Thing | Value |
|:---|:---|
| Number of words | 830,546 |
| True permutations needed for exact MinHash | 830,546! |
| Size of 830,546! | Unimaginably bigger than \(10^{2,000,000}\) |
| Is this practical? | NEVER |

In [5]:
import re
import pandas as pd
from collections import Counter

# 1. Flatten all cleaned titles into words
all_words = []
for title in titles['clean_title']:
    if pd.notna(title):
        words = title.split()
        all_words.extend(words)

# 2. Count frequencies
word_counts = Counter(all_words)

# 3. Convert to DataFrame
word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count'])

# 4. Sort by count (least frequent to most frequent)
word_counts_df = word_counts_df.sort_values(by='Count', ascending=True).reset_index(drop=True)

# 5. Save to a TXT file (word and count side by side)
with open('word_counts_sorted.txt', 'w', encoding='utf-8') as f:
    for idx, row in word_counts_df.iterrows():
        f.write(f"{row['Word']}\t{row['Count']}\n")  # tab-separated

we can try bigrams

In [9]:
import os

print("CPU count:", os.cpu_count())

CPU count: 12
... (truncated)

Try multi process

done in terminal as a py

In [16]:
import re

# 1. Clean punctuation from each title
titles['clean_title'] = titles['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# 2. Split and flatten all bigrams into one list
def get_bigrams(title):
    title = title.replace(" ", "").lower()  # remove spaces and lowercase
    return [title[i:i+2] for i in range(len(title)-1)]

all_bigrams = []
for title in titles['clean_title']:
    if pd.notna(title):  # make sure title is not NaN
        bigrams = get_bigrams(title)
        all_bigrams.extend(bigrams)

# 3. Get unique bigrams
unique_bigrams = list(set(all_bigrams))

# 4. Sort them if you want
unique_bigrams = sorted(unique_bigrams)

print(len(unique_bigrams))

9117


HERE i want to store all of the similarity matches...

In [7]:
from datasketch import MinHash, MinHashLSH
import heapq

def get_all_similar_titles(titles_df, threshold=0.5, num_perm=128):
    """
    Compute all similar title pairs using MinHash and LSH.
    
    Parameters:
    - titles_df: pd.DataFrame with column ['clean_title']
    - threshold: LSH similarity threshold
    - num_perm: number of MinHash permutations
    
    Returns:
    - List of tuples: (similarity, (title1_index, title2_index))
    """
    title_sets = titles_df['clean_title'].apply(lambda x: set(str(x).split()))
    
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    title_minhashes = {}

    # Build MinHash signatures and insert into LSH
    for idx, words in title_sets.items():
        m = MinHash(num_perm=num_perm)
        for word in words:
            m.update(word.encode('utf8'))
        lsh.insert(str(idx), m)
        title_minhashes[idx] = m

    seen_pairs = set()
    all_pairs = []

    # Query LSH and find all similar title pairs
    for idx, m in title_minhashes.items():
        candidates = lsh.query(m)
        for other_idx in candidates:
            other_idx = int(other_idx)
            if idx == other_idx:
                continue
            pair = tuple(sorted((idx, other_idx)))
            if pair in seen_pairs:
                continue
            seen_pairs.add(pair)
            sim = m.jaccard(title_minhashes[other_idx])
            all_pairs.append((sim, pair))

    # Sort all pairs by descending similarity
    all_pairs_sorted = sorted(all_pairs, reverse=True)

    return all_pairs_sorted

In [8]:
# Run it once
all_similar_pairs = get_all_similar_titles(
    titles[['clean_title']],
    threshold=0.5,
    num_perm=128
)
# I have macbook pro m3 chip: This takes 12 minutes on my machine

In [9]:
questionable_pairs = [(sim, pair) for sim, pair in all_similar_pairs if 0 <= sim <= 0.05]
for sim, (idx1, idx2) in questionable_pairs[:400]:
    title1 = titles.loc[idx1]['clean_title']
    title2 = titles.loc[idx2]['clean_title']
    print(f"Similarity: {sim:.4f}")
    print(f"Title 1: {title1}")
    print(f"Title 2: {title2}")
    print("-" * 80)

Similarity: 0.0469
Title 1: FTVOGUE Boost Converter DC 12V to DC 19V Car Power Booster Step Up Adapter Waterproof Notebook Power Supply Modification Module Regulator
Title 2: Dual USB Car Charger Lighter Adapter QC 30 54A30W 4X Rapid Faster Charging Speed for iPhone 14 13 12 11 Mini Pro Max XS X XR 8 7 6 5 iPad Pro Air Mini Galaxy Note S22 S21 S20 Ultra Plus  More
--------------------------------------------------------------------------------
Similarity: 0.0469
Title 1: MGGi DC Booster Voltage Regulator 12V Step Up to 48V 3A 145W Converter Boost Power Supply Module Boost Transformer Waterproof Adapter  for Colf Cart Club Car Truck Vehicle Boat Aluminum 12V48V
Title 2: Bestrix Car Charger Dual Port USB Quick Charge 40 5A30W Fast Charging Car USB Charger Adapter Compatible with Any iPhone14 13 12 iPadSamsung Galaxy S22 S21 S20 S10 S9 S8 Note LG Nexus
--------------------------------------------------------------------------------
Similarity: 0.0469
Title 1: ANBINGOFloor Mats Custom for 

In [10]:
questionable_pairs = [(sim, pair) for sim, pair in all_similar_pairs if 0.5 <= sim <= 0.8]

for sim, (idx1, idx2) in questionable_pairs:
    print(f"Similarity: {sim:.4f} between {idx1} and {idx2}")

Similarity: 0.7969 between 1425537 and 1425551
Similarity: 0.7969 between 1425098 and 1425336
Similarity: 0.7969 between 1424920 and 1425384
Similarity: 0.7969 between 1424920 and 1425171
Similarity: 0.7969 between 1424920 and 1424963
Similarity: 0.7969 between 1424917 and 1425487
Similarity: 0.7969 between 1424810 and 1425123
Similarity: 0.7969 between 1424696 and 1425266
Similarity: 0.7969 between 1424618 and 1424810
Similarity: 0.7969 between 1424467 and 1425376
... (truncated)

In [11]:
# Extract all title indices that appear in any pair
matched_titles = set()

for sim, (idx1, idx2) in all_similar_pairs:
    matched_titles.add(idx1)
    matched_titles.add(idx2)

print(f"Number of unique titles involved in matches: {len(matched_titles)}")

Number of unique titles involved in matches: 1117452


In [14]:
import numpy as np

def find_components(edges, n_nodes):
    parent = np.arange(n_nodes)

    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u

    def union(u, v):
        pu, pv = find(u), find(v)
        if pu != pv:
            parent[pu] = pv

    for u, v in edges:
        union(u, v)

    # Build clusters (regular Python dictionary here)
    components = {}
    for i in range(n_nodes):
        leader = find(i)
        if leader not in components:
            components[leader] = []
        components[leader].append(i)

    return list(components.values())

In [ ]:
# Build graph edges from LSH matches
edges = np.array([pair for sim, pair in all_similar_pairs])

# Get total number of nodes
max_idx = max(matched_titles)
n_nodes = max_idx + 1

# Find clusters
clusters = find_components(edges, n_nodes)
print(f"Total clusters (including singletons): {len(clusters)}")

# Filter out singleton clusters
real_clusters = [c for c in clusters if len(c) > 1]
print(f"Number of real clusters (size > 1): {len(real_clusters)}")

In [19]:
# Step 1: Filter real clusters
real_clusters = [c for c in clusters if len(c) > 1]

# Step 2: Assign cluster IDs
title_to_cluster = {}
for cluster_id, cluster in enumerate(real_clusters):
    for idx in cluster:
        title_to_cluster[idx] = cluster_id

# Step 3: Optional — Assign unmatched titles to -1
for idx in range(n_nodes):
    if idx not in title_to_cluster:
        title_to_cluster[idx] = -1

# Step 4: Add cluster labels back to your titles DataFrame
titles['cluster'] = titles.index.map(title_to_cluster)

In [20]:
# 1. Count the size of each cluster
cluster_sizes = titles['cluster'].value_counts()

# 2. Filter out cluster -1 if you assigned unmatched titles
cluster_sizes = cluster_sizes[cluster_sizes.index != -1]

# 3. View the top 20 largest clusters
print(cluster_sizes.head(20))

cluster
0.0        986464
18843.0       278
16787.0       114
33640.0       104
7770.0         94
33626.0        92
6805.0         89
33715.0        82
671.0          73
17074.0        69
42596.0        69
9472.0         68
10376.0        67
42500.0        61
17004.0        55
14421.0        54
7918.0         54
14934.0        52
868.0          52
15062.0        47
Name: count, dtype: int64


In [21]:
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

def clean_text_and_remove_stopwords(text):
    # Lowercase and remove punctuation
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Remove stopwords
    words = text.split()
    filtered_words = [w for w in words if w not in ENGLISH_STOP_WORDS]
    return " ".join(filtered_words)  # Return cleaned sentence

def get_bigrams(text):
    text = text.replace(" ", "")  # Remove spaces
    return [text[i:i+2] for i in range(len(text)-1)]

def get_all_similar_titles(titles_df, threshold=0.8, num_perm=128):
    """
    LSH on clean titles:
    - Remove stopwords
    - Generate bigrams
    - Compare with MinHash
    """
    title_sets = []

    for title in titles_df['clean_title']:
        if pd.isna(title):
            title_sets.append(set())
            continue
        cleaned_text = clean_text_and_remove_stopwords(title)  # STOPWORDS REMOVED HERE
        bigrams = get_bigrams(cleaned_text)
        title_sets.append(set(bigrams))

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    title_minhashes = {}

    for idx, bigram_set in enumerate(title_sets):
        m = MinHash(num_perm=num_perm)
        for bigram in bigram_set:
            m.update(bigram.encode('utf8'))
        lsh.insert(str(idx), m)
        title_minhashes[idx] = m

    seen_pairs = set()
    all_pairs = []

    for idx, m in title_minhashes.items():
        candidates = lsh.query(m)
        for other_idx in candidates:
            other_idx = int(other_idx)
            if idx == other_idx:
                continue
            pair = tuple(sorted((idx, other_idx)))
            if pair in seen_pairs:
                continue
            seen_pairs.add(pair)
            sim = m.jaccard(title_minhashes[other_idx])
            all_pairs.append((sim, pair))

    all_pairs_sorted = sorted(all_pairs, reverse=True)

    return all_pairs_sorted

In [22]:
all_similar_pairs = get_all_similar_titles(titles[['clean_title']], threshold=0.7, num_perm=128)

In [9]:
import numpy as np
import pandas as pd
import re
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_texts_vectorized(series):
    # Lowercase and remove punctuation using vectorized string operations
    series = series.str.lower().str.replace(r'[^\w\s]', '', regex=True)
    # Remove stopwords using a vectorized apply with set lookup
    stopwords = set(ENGLISH_STOP_WORDS)
    return series.apply(lambda s: ' '.join([w for w in s.split() if w not in stopwords]))

def get_bigrams_numpy(text):
    text = text.replace(" ", "")
    if len(text) < 2:
        return []
    chars = np.array(list(text), dtype='<U1')  # explicitly use Unicode type
    bigrams = np.char.add(chars[:-1], chars[1:])  # use np.char.add for string concatenation
    return bigrams.tolist()

def get_all_similar_titles_numpy(titles_df, threshold=0.8, num_perm=128):
    titles = titles_df['clean_title'].fillna("")

    # Vectorized cleaning
    cleaned_titles = clean_texts_vectorized(titles)

    # Vectorized bigram generation using NumPy
    bigram_sets = [set(get_bigrams_numpy(t)) for t in cleaned_titles]

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    title_minhashes = {}

    # MinHash insertion (not vectorizable due to library constraints)
    for idx, bigrams in enumerate(bigram_sets):
        m = MinHash(num_perm=num_perm)
        for b in bigrams:
            m.update(b.encode('utf8'))
        lsh.insert(str(idx), m)
        title_minhashes[idx] = m

    seen = set()
    pairs = []

    for idx, m in title_minhashes.items():
        for other in lsh.query(m):
            other = int(other)
            if idx >= other:
                continue
            pair = (idx, other)
            if pair in seen:
                continue
            seen.add(pair)
            sim = m.jaccard(title_minhashes[other])
            pairs.append((sim, pair))

    pairs.sort(reverse=True)
    return pairs

In [10]:
all_similar_pairs = get_all_similar_titles_numpy(titles[['clean_title']], threshold=0.7, num_perm=128)

In [ ]:
title_to_cluster = {}

for cluster_id, cluster in enumerate(clusters):
    for idx in cluster:
        title_to_cluster[idx] = cluster_id

# Now you can easily retrieve cluster id for any title!

IMPLEMENTING PARALELL PROCESSING

In [24]:
import multiprocessing as mp
from datasketch import MinHash, MinHashLSH
import pandas as pd

def compute_minhash(idx_words_tuple, num_perm=128):
    idx, words = idx_words_tuple
    m = MinHash(num_perm=num_perm)
    for word in words:
        m.update(word.encode('utf8'))
    return (idx, m)

def get_all_similar_titles_parallel(titles_df, threshold=0.5, num_perm=128):
    title_sets = titles_df['clean_title'].apply(lambda x: set(str(x).split()))
    title_sets = list(title_sets.items())

    # Parallel MinHash computation
    with mp.Pool(processes= 8) as pool:
        results = pool.starmap(compute_minhash, [(idx, words) for idx, words in title_sets])

    title_minhashes = dict(results)
    
    # Build LSH
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for idx, m in title_minhashes.items():
        lsh.insert(str(idx), m)

    # Find all pairs
    seen_pairs = set()
    all_pairs = []
    for idx, m in title_minhashes.items():
        candidates = lsh.query(m)
        for other_idx in candidates:
            other_idx = int(other_idx)
            if idx == other_idx:
                continue
            pair = tuple(sorted((idx, other_idx)))
            if pair in seen_pairs:
                continue
            seen_pairs.add(pair)
            sim = m.jaccard(title_minhashes[other_idx])
            all_pairs.append((sim, pair))

    all_pairs_sorted = sorted(all_pairs, reverse=True)
    return all_pairs_sorted

In [25]:
all_similar_pairs = get_all_similar_titles_parallel(
    titles[['clean_title']],
    threshold=0.5,
    num_perm=128
)

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'compute_minhash' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task =

KeyboardInterrupt: 

In [26]:
print('hi')

hi


In [ ]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
import multiprocessing as mp

# Step 1: Define compute_minhash at top level
def compute_minhash(idx, words, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in words:
        m.update(word.encode('utf8'))
    return (idx, m)

# Step 2: Define main function
def get_all_similar_titles_parallel(titles_df, threshold=0.5, num_perm=128, num_workers=None):
    title_sets = titles_df['clean_title'].apply(lambda x: set(str(x).split()))
    title_sets = list(title_sets.items())

    if num_workers is None:
        num_workers = mp.cpu_count()

    with mp.Pool(processes=num_workers) as pool:
        results = pool.starmap(compute_minhash, [(idx, words, num_perm) for idx, words in title_sets])

    title_minhashes = dict(results)

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for idx, m in title_minhashes.items():
        lsh.insert(str(idx), m)

    seen_pairs = set()
    all_pairs = []
    for idx, m in title_minhashes.items():
        candidates = lsh.query(m)
        for other_idx in candidates:
            other_idx = int(other_idx)
            if idx == other_idx:
                continue
            pair = tuple(sorted((idx, other_idx)))
            if pair in seen_pairs:
                continue
            seen_pairs.add(pair)
            sim = m.jaccard(title_minhashes[other_idx])
            all_pairs.append((sim, pair))

    all_pairs_sorted = sorted(all_pairs, reverse=True)
    return all_pairs_sorted

# Step 3: Wrap execution inside main
if __name__ == '__main__':
    # Example usage
    import pandas as pd
    import re
    import time

    titles = pd.read_csv('amazon_products.csv')

    # Must clean here
    titles['clean_title'] = titles['title'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

    start_time = time.time()

    # Now this will find clean_title and work:
    all_similar_pairs = get_all_similar_titles_parallel(
        titles[['clean_title']],
        threshold=0.5,
        num_perm=128,
        num_workers=8
    )

    print(f"Found {len(all_similar_pairs)} pairs.")
    print(f"Total time: {time.time() - start_time:.2f} seconds")

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'compute_minhash' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task =

: 